# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [2]:
from collections import Counter
import math

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import torchtext
from torchtext.data.utils import get_tokenizer

import helper

In [3]:
def get_local_device() -> torch.device:
    available_device = "cuda" if torch.cuda.is_available() else "cpu"
    return torch.device(available_device)

device = get_local_device()
print(device)

cuda


In [ ]:
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)

train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

In [ ]:
def tensor_to_list(tensor: torch.Tensor) -> list:
    return tensor.cpu().detach().numpy().tolist()


def scores_to_top_tokens(scores: torch.Tensor) -> list:
    _, top_index_tensor = torch.topk(scores, k=1)
    top_index_tensor.squeeze_().t_()
    top_index_list = tensor_to_list(top_index_tensor)
    return top_index_list


def tokens_to_words(batches: list) -> list:
    decoded_batches = []
    for batch in batches:
        decoded_batch = [TEXT.vocab.itos[token] for token in batch]
        decoded_batches.append(decoded_batch)
    return decoded_batches


def decode_transformer_output(scores: torch.Tensor) -> list:
    top_tokens = scores_to_top_tokens(scores)
    predicted_words = tokens_to_words(top_tokens)
    return predicted_words


def decode_targets(targets: torch.Tensor, batch_size: int) -> list:
    unsquashed_targets = targets.view(-1, batch_size).t()
    target_tokens = tensor_to_list(unsquashed_targets)
    return tokens_to_words(target_tokens)


def decode_inputs(inputs: torch.Tensor) -> list:
    input_tokens = tensor_to_list(inputs.t())
    return tokens_to_words(input_tokens)


def join_sequences(decoded_sequences: list) -> list:
    return [" ".join(sequence) for sequence in decoded_sequences]


def present_result(model, inputs: torch.Tensor, targets: torch.Tensor, index: int) -> None:
    model.eval()
    output = model(inputs)
    input_text = join_sequences(decode_inputs(inputs))[index]
    target_text = join_sequences(decode_targets(targets, 20))[index]
    output_text = join_sequences(decode_transformer_output(output))[index]
    print( "INPUT\n-----\n" 
          f"{input_text}\n\n"
           "TARGET\n------\n"
          f"{target_text}\n\n"
           "OUTPUT\n------\n"
          f"{output_text}\n")

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [5]:
view_line_range = (100100, 100110)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
word_count_line = [count for count in word_count_line if count != 0]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))
print('Median number of words in each line: {}'.format(np.median(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 11.088582100483375
Median number of words in each line: 8.0

The lines 100100 to 100110:
elaine: well, it's funny. i mean, you have a big head. or is it 'cause of your neck? 

jerry: no, i think the head does most of the stretching. 

george: regardless. i had to walk around for the rest of the party in some cheap metlife windbreaker. now, it is payback time. 

elaine: i really think it's the size of your neck. 

george: it's my head! 



---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [6]:
def get_unique_words_sorted_by_frequency(text: list) -> list:
    word_counts = Counter(text)
    sorted_counts = word_counts.most_common()
    sorted_words = dict(sorted_counts).keys()
    return sorted_words


def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocabulary = get_unique_words_sorted_by_frequency(text)
    int_to_vocab = dict(enumerate(vocabulary))
    vocab_to_int = dict(zip(int_to_vocab.values(), int_to_vocab.keys()))
    return (vocab_to_int, int_to_vocab)

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [7]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    punctuation_tokens = {'.': "<PERIOD>",
                          ',': "<COMMA>",
                          '"': "<QUOTE>",
                          ';': "<SEMICOLON>",
                          '!': "<EXCLAMATION>",
                          '?': "<QUESTION>",
                          '(': "<LPARANTHESES>",
                          ')': "<RPARANTHESES>",
                          '-': "<DASH>",
                          '\n': "<RETURN>"}
    return punctuation_tokens

## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [8]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [10]:
import helper

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [11]:
def get_next_data_sequence(words: list, sequence_length: int) -> (list, int):
    word_count = len(words)
    scan_index = 0
    last_scan_index = word_count - sequence_length -1
    
    while scan_index <= last_scan_index:
        features = words[scan_index:scan_index+sequence_length]
        target = words[scan_index+sequence_length]
        scan_index += 1
        yield features, target


def build_data_tensors(words: list, sequence_length: int) -> (torch.Tensor, torch.Tensor):
    features_list, targets_list = [], []
    for features, targets in get_next_data_sequence(words, sequence_length):
        features_list.append(features)
        targets_list.append(targets)
        
    features_tensor = torch.LongTensor(features_list)
    targets_tensor = torch.LongTensor(targets_list)
    return features_tensor, targets_tensor


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    features, targets = build_data_tensors(words, sequence_length)
    data = TensorDataset(features, targets)
    loader = DataLoader(dataset=data, batch_size=batch_size, shuffle=True)
    return loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own

data = int_text[:12]
print(data)
gen = get_next_data_sequence(data, 10)
print(next(gen))
print(next(gen))
build_data_tensors(data, 10)

[24, 22, 47, 1, 1, 1, 17, 47, 22, 82, 20, 6]
([24, 22, 47, 1, 1, 1, 17, 47, 22, 82], 20)
([22, 47, 1, 1, 1, 17, 47, 22, 82, 20], 6)


(tensor([[24, 22, 47,  1,  1,  1, 17, 47, 22, 82],
         [22, 47,  1,  1,  1, 17, 47, 22, 82, 20]]),
 tensor([20,  6]))

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [12]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[16, 17, 18, 19, 20],
        [ 8,  9, 10, 11, 12],
        [35, 36, 37, 38, 39],
        [15, 16, 17, 18, 19],
        [43, 44, 45, 46, 47],
        [31, 32, 33, 34, 35],
        [32, 33, 34, 35, 36],
        [ 0,  1,  2,  3,  4],
        [30, 31, 32, 33, 34],
        [27, 28, 29, 30, 31]])

torch.Size([10])
tensor([21, 13, 40, 20, 48, 36, 37,  5, 35, 32])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [12]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super().__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=output_size)
        self.fc_dropout = nn.Dropout(p=dropout)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        long_int_input = nn_input.type(torch.LongTensor).to(self.device)
        embedded_input = self.embed(long_int_input)
        lstm_outputs, new_hidden = self.lstm(embedded_input, hidden)
        last_lstm_output = lstm_outputs[:,-1,:]
        last_lstm_output = self.fc_dropout(last_lstm_output)
        decoded_output = self.fc(last_lstm_output)
        
        return decoded_output, new_hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        state_size = (self.lstm.num_layers, batch_size, self.lstm.hidden_size)
        h = torch.zeros(size=state_size, device=self.device)
        c = torch.zeros(size=state_size, device=self.device)
        hidden_state = (h,c)
        return hidden_state

In [13]:
class PositionalEncoding(nn.Module):
    def __init__(self, dimension: int, dropout: int=0.1, max_length: int=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        positional_values = self.__generate_position_values(dimension, max_length)
        self.register_buffer("positional_values", positional_values)
        
    @staticmethod
    def __generate_position_values(dimension: int, max_length: int):
        values = torch.zeros(max_length, dimension)
        positions = torch.arange(0, max_length, dtype=torch.float)
        positions = positions.unsqueeze(1)
        
        scaling_steps = torch.arange(0, dimension, 2).float()
        scaling = torch.exp(scaling_steps * (-math.log(10000.0)/dimension))
        
        values[:, 0::2] = torch.sin(positions * scaling)
        values[:, 1::2] = torch.cos(positions * scaling)
        values = values.unsqueeze(0).transpose(0, 1)
        
        return values
    
    def forward(self, x):
        x = x + self.positional_values[:x.size(0), :]
        return self.dropout(x)

In [15]:
class TransformerScriptGenerator(nn.Module):
    def __init__(self,
                 vocabulary_size: int,
                 embedding_dim: int,
                 attention_head_count: int,
                 encoder_fc_dim: int,
                 encoder_layer_count: int,
                 dropout: float=0.5) -> None:
        
        super().__init__()
        
        self.input_mask = None
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        self.embedding_scale = math.sqrt(embedding_dim)
        self.positional_encoder = PositionalEncoding(embedding_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(embedding_dim, attention_head_count, encoder_fc_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, encoder_layer_count)
        self.decoder = nn.Linear(embedding_dim, vocabulary_size)
        
        self.__init_weights()
        
    def __init_weights(self) -> None:
        value_range = 0.1
        self.embedding.weight.data.uniform_(-value_range, value_range)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-value_range, value_range)
        
    @staticmethod
    def __generate_input_mask(mask_size: int) -> torch.Tensor():
        mask = torch.ones(mask_size, mask_size, dtype=bool)
        mask = torch.triu(mask).t().float()
        mask = mask.masked_fill(mask == 0, float('-inf'))
        mask = mask.masked_fill(mask == 1, 0.0)
        return mask
    
    @staticmethod
    def __get_output_for_last_word(full_output: torch.Tensor) -> torch.Tensor:
        return full_output[:,-1,:]
    
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        mask_size = input.shape[0]
        if self.input_mask is None or self.input_mask.size(0) != mask_size:
            self.input_mask = self.__generate_input_mask(mask_size).to(device)
            
        input = self.embedding(input) * self.embedding_scale
        input = self.positional_encoder(input)
        output = self.transformer_encoder(input, self.input_mask)
        output = self.__get_output_for_last_word(output)
        output = self.decoder(output)
        return output

In [16]:
def transformer_training_step(model: TransformerScriptGenerator, 
                              optimizer: nn.Module, 
                              criterion: nn.Module,
                              input: torch.Tensor, 
                              target: torch.Tensor) -> torch.Tensor:
    optimizer.zero_grad()
    input_batch_second = input.to(device).t()
    output = model(input_batch_second)
    loss = criterion(output, target.to(device))
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 2.0)
    optimizer.step()
    return loss

In [17]:
model = TransformerScriptGenerator(1000, 128, 4, 1024, 5).to(device)
print(sample_x)
model(sample_x.t().to(device))

tensor([[16, 17, 18, 19, 20],
        [ 8,  9, 10, 11, 12],
        [35, 36, 37, 38, 39],
        [15, 16, 17, 18, 19],
        [43, 44, 45, 46, 47],
        [31, 32, 33, 34, 35],
        [32, 33, 34, 35, 36],
        [ 0,  1,  2,  3,  4],
        [30, 31, 32, 33, 34],
        [27, 28, 29, 30, 31]])


tensor([[-0.0872,  0.6260,  0.6056,  ...,  0.1357,  0.2484,  0.1315],
        [ 0.0206, -0.1255,  1.4584,  ..., -1.4299,  0.1306,  0.9725],
        [-0.0759,  1.4677,  0.5366,  ..., -0.6991,  0.1620,  1.2998],
        [ 0.1329,  0.0469, -0.3864,  ..., -0.6919, -0.6370,  1.1347],
        [ 0.2470, -0.0369, -0.0920,  ...,  0.8227, -1.1005,  0.1109]],
       device='cuda:0', grad_fn=<AddmmBackward>)

## Neural Network Training

In [18]:
def should_skip_batch(input_batch: torch.Tensor, batch_size: int) -> bool:
    if input_batch.shape[1] != batch_size:
        print("Invalid batch size; skipping.")
        return True
    return False


def train_transformer(model: TransformerScriptGenerator, 
                      optimizer: nn.Module, 
                      criterion: nn.Module,
                      lr_adjuster: nn.Module,
                      epoch_count: int,
                      train_loader: DataLoader,
                      batch_size: int) -> None:
    
    model.train()
    print(f"Training for {epoch_count} epochs...")
    
    for epoch in range(epoch_count):
        print(f"Epoch {epoch}")
        total_batches = 0
        total_loss = 0
        
        for inputs, targets in train_loader:
            inputs.t_()
            if should_skip_batch(inputs, batch_size):
                continue
            total_batches += 1
            total_loss += transformer_training_step(model, optimizer, criterion, inputs, targets)
            if total_batches%500 == 0:
                print(f"+ Average training loss: {total_loss / total_batches} for {total_batches} batches")
         
        lr_adjuster.step()
        print(f"+ Average training loss: {total_loss / total_batches}")
    

In [21]:
model = TransformerScriptGenerator(vocabulary_size=len(set(int_text))+1, # the extra word is the padding token
                                   embedding_dim=256, 
                                   attention_head_count=4, 
                                   encoder_fc_dim=256, 
                                   encoder_layer_count=3,
                                   dropout=0.2).to(device)

sequence_length = 30
batch_size = 30
train_loader = batch_data(int_text, sequence_length, batch_size)

epoch_count = 1
learning_rate = 0.001#5.0
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)#torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [ ]:
train_transformer(model, optimizer, criterion, scheduler, epoch_count, train_loader, batch_size)

Training for 1 epochs...
Epoch 0
+ Average training loss: 6.0376386642456055 for 500 batches
+ Average training loss: 5.866919040679932 for 1000 batches
+ Average training loss: 5.779430866241455 for 1500 batches
+ Average training loss: 5.725190162658691 for 2000 batches
+ Average training loss: 5.705448627471924 for 2500 batches
+ Average training loss: 5.680546283721924 for 3000 batches
+ Average training loss: 5.685174465179443 for 3500 batches
+ Average training loss: 5.681693077087402 for 4000 batches
+ Average training loss: 5.6712846755981445 for 4500 batches
+ Average training loss: 5.66264533996582 for 5000 batches
+ Average training loss: 5.652032375335693 for 5500 batches
+ Average training loss: 5.647731781005859 for 6000 batches
+ Average training loss: 5.651260852813721 for 6500 batches
+ Average training loss: 5.650496006011963 for 7000 batches
+ Average training loss: 5.651313781738281 for 7500 batches
+ Average training loss: 5.655027389526367 for 8000 batches
+ Avera

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [38]:
def generate_with_transformer(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        current_seq = torch.LongTensor(current_seq).to(device)
                
        # get the output of the rnn
        output = rnn(current_seq)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data.cpu()
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [ ]:
gen_length = 400 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate_with_transformer(model, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

In [19]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()